In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt

# 创建示例数据
ag_df = pd.read_excel('data/ag_df.xlsx')

ag_df

,Unnamed: 0,date,open,high,low,close,volume,hold,settle,buy_sell_rate,fund_op,fund_sell_op
0,0,2012-05-10,6190,6196,6082,6142,259996,33726,6126,0.260501,NaN,NaN
1,1,2012-05-11,6110,6115,6000,6002,206804,55020,6056,-0.899700,NaN,NaN
2,2,2012-05-14,6040,6045,5960,5977,207668,57028,5999,-0.368078,NaN,NaN
3,3,2012-05-15,5895,5910,5711,5843,445150,82238,5813,0.513435,NaN,NaN
4,4,2012-05-16,5750,5786,5643,5647,599186,108376,5704,-1.009386,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3005,3005,2024-09-09,7260,7265,6884,6977,1971643,499827,7032,-0.788304,NaN,NaN
3006,3006,2024-09-10,7006,7062,6949,7055,1084221,505043,7008,0.666194,NaN,NaN
3007,3007,2024-09-11,7075,7179,6941,7177,1258139,499151,7031,2.034276,BUY,NaN
3008,3008,2024-09-12,7109,7187,7023,7157,1374324,486857,7108,0.684644,NaN,NaN


In [3]:
ag_df = ag_df[ag_df["settle"] != 0]
ag_df

,Unnamed: 0,date,open,high,low,close,volume,hold,settle,buy_sell_rate,fund_op,fund_sell_op
0,0,2012-05-10,6190,6196,6082,6142,259996,33726,6126,0.260501,NaN,NaN
1,1,2012-05-11,6110,6115,6000,6002,206804,55020,6056,-0.899700,NaN,NaN
2,2,2012-05-14,6040,6045,5960,5977,207668,57028,5999,-0.368078,NaN,NaN
3,3,2012-05-15,5895,5910,5711,5843,445150,82238,5813,0.513435,NaN,NaN
4,4,2012-05-16,5750,5786,5643,5647,599186,108376,5704,-1.009386,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3005,3005,2024-09-09,7260,7265,6884,6977,1971643,499827,7032,-0.788304,NaN,NaN
3006,3006,2024-09-10,7006,7062,6949,7055,1084221,505043,7008,0.666194,NaN,NaN
3007,3007,2024-09-11,7075,7179,6941,7177,1258139,499151,7031,2.034276,BUY,NaN
3008,3008,2024-09-12,7109,7187,7023,7157,1374324,486857,7108,0.684644,NaN,NaN


In [4]:
ag_df.drop("Unnamed: 0", axis=1, inplace=True)
# ag_df.set_index("date", inplace=True)
ag_df

C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_1824\2912393222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ag_df.drop("Unnamed: 0", axis=1, inplace=True)


,date,open,high,low,close,volume,hold,settle,buy_sell_rate,fund_op,fund_sell_op
0,2012-05-10,6190,6196,6082,6142,259996,33726,6126,0.260501,NaN,NaN
1,2012-05-11,6110,6115,6000,6002,206804,55020,6056,-0.899700,NaN,NaN
2,2012-05-14,6040,6045,5960,5977,207668,57028,5999,-0.368078,NaN,NaN
3,2012-05-15,5895,5910,5711,5843,445150,82238,5813,0.513435,NaN,NaN
4,2012-05-16,5750,5786,5643,5647,599186,108376,5704,-1.009386,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3005,2024-09-09,7260,7265,6884,6977,1971643,499827,7032,-0.788304,NaN,NaN
3006,2024-09-10,7006,7062,6949,7055,1084221,505043,7008,0.666194,NaN,NaN
3007,2024-09-11,7075,7179,6941,7177,1258139,499151,7031,2.034276,BUY,NaN
3008,2024-09-12,7109,7187,7023,7157,1374324,486857,7108,0.684644,NaN,NaN


In [9]:
def strategy(df):
    df = ag_df.copy()[["date","buy_sell_rate"]]
    # 查出满足 -1>buy_sell_rate 或者 buy_sell_rate>1.3 的记录
    # filtered_df = ag_df[(ag_df['buy_sell_rate'] < -1) | (ag_df['buy_sell_rate'] > 1.3)]
    df['op'] = df['buy_sell_rate'].apply(lambda x: 'buy' if x > 1.3 else ('sell' if x < -1 else None))
    filtered_df = df[df['op'].notnull()]
    
    return filtered_df
filtered_df =strategy(ag_df)

In [25]:
# 假设 filtered_df 是你的 DataFrame
# 分成两组
buy_df = filtered_df[filtered_df['op'] == 'buy'].copy()
sell_df = filtered_df[filtered_df['op'] == 'sell'].copy()
sell_df.reset_index(drop=True, inplace=True)
sell_df

,date,buy_sell_rate,op
0,2012-05-16,-1.009386,sell
1,2013-03-27,-1.009928,sell
2,2013-04-17,-1.007557,sell
3,2013-04-23,-1.438849,sell
4,2013-04-26,-1.064483,sell
...,...,...,...
121,2024-07-19,-1.771064,sell
122,2024-07-25,-3.047091,sell
123,2024-08-06,-1.143767,sell
124,2024-08-28,-1.129268,sell


In [31]:
buy_df

,date,buy_sell_rate,op,sell_date
308,2013-08-15,3.479576,buy,2013-10-30
314,2013-08-23,2.560426,buy,2013-10-30
615,2014-11-17,2.732558,buy,2015-01-22
660,2015-01-21,1.388175,buy,2015-04-09
691,2015-03-12,1.332175,buy,2015-05-26
...,...,...,...,...
2977,2024-07-31,1.312441,buy,2024-09-02
2985,2024-08-12,1.500625,buy,NaT
2988,2024-08-15,1.529052,buy,NaT
2995,2024-08-26,1.354872,buy,NaT


In [30]:
sell_dates= sell_df['date']
# 从小到大排序
sell_dates.sort_values()
def cal_sell_date(buy_date):
    # return sell_df[sell_dates >= buy_date + pd.Timedelta(days=30)]['date'].values[0]
    # Filter the DataFrame first
    filtered_dates = sell_df[sell_dates >= buy_date + pd.Timedelta(days=30)]['date'].values
    
    # Check if the filtered array is not empty
    if filtered_dates.size > 0:
        return filtered_dates[0]
    else:
        return None  # or handle the case where no valid sell date is found
    

# 添加 sell_date 列
buy_df['sell_date'] = buy_df['date'].apply(cal_sell_date)
buy_df

,date,buy_sell_rate,op,sell_date
308,2013-08-15,3.479576,buy,2013-10-30
314,2013-08-23,2.560426,buy,2013-10-30
615,2014-11-17,2.732558,buy,2015-01-22
660,2015-01-21,1.388175,buy,2015-04-09
691,2015-03-12,1.332175,buy,2015-05-26
...,...,...,...,...
2977,2024-07-31,1.312441,buy,2024-09-02
2985,2024-08-12,1.500625,buy,NaT
2988,2024-08-15,1.529052,buy,NaT
2995,2024-08-26,1.354872,buy,NaT
